In [2]:
import requests
import pandas as pd

#OVERPASS_URL = "https://overpass-api.de/api/interpreter"
OVERPASS_URL = "https://overpass.openstreetmap.ru/api/interpreter"

def get_bars_regensburg_df(center_lat=49.019533, center_lon=12.097487, radius_m=1200):
    query = f"""
    [out:json][timeout:60];
    (
      nwr["amenity"="bar"](around:{radius_m},{center_lat},{center_lon});
      nwr["amenity"="pub"](around:{radius_m},{center_lat},{center_lon});
    );
    out center tags;
    """
    r = requests.get(OVERPASS_URL, params={"data": query}, timeout=90)
    r.raise_for_status()
    data = r.json()

    rows = []
    for el in data.get("elements", []):
        tags = el.get("tags", {})
        lat = el.get("lat") or el.get("center", {}).get("lat")
        lon = el.get("lon") or el.get("center", {}).get("lon")
        rows.append({
            "name": tags.get("name"),
            #"amenity": tags.get("amenity"),
            "lat": lat,
            "lon": lon,
            "opening_hours": tags.get("opening_hours"),
            #"website": tags.get("website") or tags.get("contact:website"),
            #"phone": tags.get("phone") or tags.get("contact:phone"),
            "street": tags.get("addr:street"),
            "housenumber": tags.get("addr:housenumber"),
            "postcode": tags.get("addr:postcode"),
            "city": tags.get("addr:city"),
            #"osm_type": el.get("type"),
            #"osm_id": el.get("id"),
        })

    df = pd.DataFrame(rows).dropna(subset=["lat","lon"]).drop_duplicates(subset=["osm_type","osm_id"])
    return df

# Beispiel:
df = get_bars_regensburg_df(radius_m=1200)
df.head()


ConnectTimeout: HTTPSConnectionPool(host='overpass.openstreetmap.ru', port=443): Max retries exceeded with url: /api/interpreter?data=%0A++++%5Bout%3Ajson%5D%5Btimeout%3A60%5D%3B%0A++++%28%0A++++++nwr%5B%22amenity%22%3D%22bar%22%5D%28around%3A1200%2C49.019533%2C12.097487%29%3B%0A++++++nwr%5B%22amenity%22%3D%22pub%22%5D%28around%3A1200%2C49.019533%2C12.097487%29%3B%0A++++%29%3B%0A++++out+center+tags%3B%0A++++ (Caused by ConnectTimeoutError(<HTTPSConnection(host='overpass.openstreetmap.ru', port=443) at 0x7d816e1252d0>, 'Connection to overpass.openstreetmap.ru timed out. (connect timeout=90)'))

In [ ]:
#save the data as a backup cause of timeout errors 
df.to_csv("regensburg_bars_backup.csv", index=False)

Luftlinie zwischen zwei Objekten (User oder Bars)

In [3]:
import math




def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000  # meters
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.asin(math.sqrt(a))




In [ ]:
user_lat, user_lon = 49.019533, 12.097487
bar_lat, bar_lon = 49.0208, 12.0959

dist_m = haversine_m(user_lat, user_lon, bar_lat, bar_lon)
print(f"Distanz: {dist_m:.0f} m")


Distanz: 182 m


Aus Adresse machen wir long, lat für haversine_m

In [7]:
def geocode_address(address: str):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": address,
        "format": "json",
        "limit": 1,
    }
    headers = {
        "User-Agent": "pubcrawl-uni-app/1.0 (contact: d.sowada99@gmail.com)"
    }
    r = requests.get(url, params=params, headers=headers, timeout=30)
    r.raise_for_status()
    data = r.json()
    if not data:
        return None
    return float(data[0]["lat"]), float(data[0]["lon"])

# Beispiel:
lat, lon = geocode_address("Neupfarrplatz 1, 93047 Regensburg")
print(lat)
print(lon)

49.0183411
12.0963412


Workflow generieren:

In [9]:
#Adresse
bar = "Neupfarrplatz 1, 93047 Regensburg"
user = "Im Anger 89, 93098 Mintraching"

bar_lat, bar_long = geocode_address(bar)
user_lat, user_long = geocode_address(user)

print(haversine_m(bar_lat, bar_long,user_lat, user_long ))


13017.033553475068
